<center><h2>Kaggle - Taxi Fare Prediction</h2></center>

To solve the regression problem asked I have started using linear, the SVR method, the KNN regressor follwed by the ensemble methods. Linear did not provide good result. KNN/SVR did not work well as the machine was hanging with large data points for KNN/SVR, so dropped that approach. Next I started trying Random Tree Regressor with default parameters and acheived a score of 4.06963 followed by Random Tree Regressor with random seed, then with 1000 estimators count, then max_depth as 10 and estimators_max_featues as sqrt then log2 and achieved scores as 4.05886, 4.03511, 3.88248, 3.87734 respectively. <br> Next I chose XGBRegressor and got a score of 3.75939 with no max depth and 3.75389 with max depth defined and at last all the default settings and got best score of 3.70257. At last I tried Gradient Boosting Regressor and achieved score of 3.71139.

In [1]:
# Import all the needed libraries and the data from the file.
import numpy as np
import pandas as pd
data = pd.read_csv("./new-york-city-taxi-fare-prediction/train.csv")
data.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [2]:
# Take 3M points for training.
data = data.head(300000)

In [3]:
# Split the date_timestamp column to year, month, dayofweek, and hour.
data['year'] = pd.to_datetime(data['pickup_datetime']).dt.year
data['month'] = pd.to_datetime(data['pickup_datetime']).dt.month
data['day'] = pd.to_datetime(data['pickup_datetime']).dt.dayofweek
data['time'] = pd.to_datetime(data['pickup_datetime']).dt.hour
data.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,time
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1,2009,6,0,17
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,1,16
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,3,0
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,5,4
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,1,7


In [4]:
# Drop the column pickup_datetime.
data = data.drop(columns=['pickup_datetime'])
data.head()

,key,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,time
0,2009-06-15 17:26:21.0000001,4.5,-73.844311,40.721319,-73.841610,40.712278,1,2009,6,0,17
1,2010-01-05 16:52:16.0000002,16.9,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,1,16
2,2011-08-18 00:35:00.00000049,5.7,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,3,0
3,2012-04-21 04:30:42.0000001,7.7,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,5,4
4,2010-03-09 07:51:00.000000135,5.3,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,1,7


In [5]:
# Define the function to calculate the distance between two locations.
from math import sin, cos, sqrt, atan2
def giveDistance(latitude_1, longitude_1, latitude_2,longitude_2):
    # Radius of earth in kms
    radius_of_earth = 6373.0

    # Difference between the lats and lons.
    diff_longitudes = longitude_2 - longitude_1
    diff_latitudes = latitude_2 - latitude_1

    s = sin(diff_latitudes / 2)**2 + cos(latitude_1) * cos(latitude_2) * sin(diff_longitudes / 2)**2
    t = 2 * atan2(sqrt(s), sqrt(1 - s))
    # Return the distance.
    return (radius_of_earth * t)

In [6]:
# Calculate distance for all the 3M data points and add a new column distance to the data.
distance = []
for i in range(300000):
    distance.append(giveDistance(data.iloc[i,3], data.iloc[i,2], data.iloc[i,5], data.iloc[i,4]))
data['distance'] = distance

In [7]:
# Drop the locations column that are columns=['pickup_longitude','pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']
data = data.drop(columns=['pickup_longitude','pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'])

In [8]:
# Final look of the dataset after all the preprocessing.
data.head()

,key,fare_amount,passenger_count,year,month,day,time,distance
0,2009-06-15 17:26:21.0000001,4.5,1,2009,6,0,17,60.096950
1,2010-01-05 16:52:16.0000002,16.9,1,2010,1,1,16,507.400241
2,2011-08-18 00:35:00.00000049,5.7,2,2011,8,3,0,87.023524
3,2012-04-21 04:30:42.0000001,7.7,1,2012,4,5,4,161.472386
4,2010-03-09 07:51:00.000000135,5.3,1,2010,3,1,7,123.988885


In [9]:
# Remove the data entries that have fare as null.
null_columns=data.columns[data.isnull().any()]
data[null_columns].isnull().sum()
data.dropna(inplace=True)

In [10]:
# Get the numpy array.
X = np.array (data)
Y = X[:,1]
X = X[:,2:]

In [12]:
from sklearn.linear_model import LinearRegression
# Linear Regression.
clf = LinearRegression().fit(X, Y)

In [ ]:
from sklearn.svm import SVR
# SVR Regression.
clf = SVR(gamma='scale', C=1.0, epsilon=0.2)
clf.fit(X, Y) 

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
# KNN Regressor.
clf = KNeighborsRegressor(n_neighbors=10)
clf.fit(X, Y) 

In [43]:
from sklearn.ensemble import RandomForestRegressor
# Random Forest Regressor
clf = RandomForestRegressor(n_estimators = 1000, max_features='log2', max_depth=10, n_jobs=-1)
clf.fit(X,Y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='log2', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [14]:
from sklearn import ensemble
# GradientBoostingRegressor
clf = ensemble.GradientBoostingRegressor(n_estimators=250)
clf.fit(X,Y)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=250, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [51]:
from xgboost import XGBRegressor
# XGBRegressor
clf = classifier = XGBRegressor(silent=False,
                      early_stopping_rounds = 4,
                      n_jobs=-1, 
                      n_estimators=1000)
clf.fit(X, Y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, early_stopping_rounds=4, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=-1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=False, subsample=1)

In [24]:
# Preprocess the test data same as train data.
test_data = pd.read_csv("./new-york-city-taxi-fare-prediction/test.csv")
test_data['year'] = pd.to_datetime(test_data['pickup_datetime']).dt.year
test_data['month'] = pd.to_datetime(test_data['pickup_datetime']).dt.month
test_data['day'] = pd.to_datetime(test_data['pickup_datetime']).dt.dayofweek
test_data['time'] = pd.to_datetime(test_data['pickup_datetime']).dt.hour
test_data = test_data.drop(columns=['pickup_datetime'])
test_data.head()

,key,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,time
0,2015-01-27 13:08:24.0000002,-73.973320,40.763805,-73.981430,40.743835,1,2015,1,1,13
1,2015-01-27 13:08:24.0000003,-73.986862,40.719383,-73.998886,40.739201,1,2015,1,1,13
2,2011-10-08 11:53:44.0000002,-73.982524,40.751260,-73.979654,40.746139,1,2011,10,5,11
3,2012-12-01 21:12:12.0000002,-73.981160,40.767807,-73.990448,40.751635,1,2012,12,5,21
4,2012-12-01 21:12:12.0000003,-73.966046,40.789775,-73.988565,40.744427,1,2012,12,5,21


In [25]:
# Get the distance and add a new distance column and delete irrelevant columns.
distance = []
for i in range(test_data.shape[0]):
    distance.append(giveDistance(test_data.iloc[i,2], test_data.iloc[i,1], test_data.iloc[i,4], test_data.iloc[i,3]))
test_data['distance'] = distance
test_data = test_data.drop(columns=['pickup_longitude','pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'])
test_data.head()

,key,passenger_count,year,month,day,time,distance
0,2015-01-27 13:08:24.0000002,1,2015,1,1,13,137.289550
1,2015-01-27 13:08:24.0000003,1,2015,1,1,13,147.478190
2,2011-10-08 11:53:44.0000002,1,2011,10,5,11,37.374260
3,2012-12-01 21:12:12.0000002,1,2012,12,5,21,118.755873
4,2012-12-01 21:12:12.0000003,1,2012,12,5,21,322.496233


In [34]:
# Delete the rows that have null values.
null_columns=test_data.columns[test_data.isnull().any()]
test_data[null_columns].isnull().sum()
test_data.dropna(inplace=True)
X_test_complete = np.array (test_data)
X_test = X_test_complete[:,1:]
X_test

array([[1, 2015, 1, 1, 13, 137.28955002692095],
       [1, 2015, 1, 1, 13, 147.47818959810158],
       [1, 2011, 10, 5, 11, 37.37425975098158],
       ...,
       [6, 2015, 4, 6, 20, 1368.3329344010906],
       [6, 2015, 1, 5, 1, 515.2315436693702],
       [6, 2015, 1, 6, 14, 84.49434499737491]], dtype=object)

In [52]:
# Predict the output and write in the file.
Y_test = clf.predict(X_test)
# Write the results on a file.
import csv
with open('submission.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    lines = []
    lines.append(['key', 'fare_amount'])
    for i in range(len(Y_test)):
        lines.append([X_test_complete[i,0],Y_test[i]])
    writer.writerows(lines)
    writeFile.close()

<h3>Best 2 scores : </h3><hr> First is <b>3.70257</b> by XGBooster Regressor and Second one is <b>3.71139</b> by Gradient Booster Regressor.<br/>
XGBoosterRegressor is currently one of the state-of-the-art boosting algorithm for Regression as well as Classification. Its fast as written in C++ andis parallelizable and has wide variety of tunning parameters. It stands for Extreme Gradient Boosting.<br/>
GradientBoostingClassifier is predefined sklearn boosting method, models in forward stagewise fashion, with multiple oss function and different parameters.

<h3>Best 2 methods : </h3><hr/> The difference among all 3 methods tried is less so cannot comment and claim that the best best scores acheived are the best rather if tunned more possiblity is that Random Tree or Gradient Booster could have performed better than XGBooster. It depends upon better tunning and application. Here I can say that definately ensemble methods perform better than and other kernel/linear/kNN regressor. KNN is applied could have taken a lot of time same as SVR was taking. And linear did not work well here.